# Generate Ballerina Solutions for Codeforces Problems
This notebook uses Claude API to generate Ballerina code solutions for problems in ballerina_grpo_X4.json with:
- 2 retry attempts with feedback
- Resuming capability (skip already processed entries)
- Test case validation from Demo Input/Output


In [ ]:
import json
import anthropic
import os
import subprocess
import tempfile
import re
import ast
from pathlib import Path
from datetime import datetime

# Configuration
API_KEY = "<REDACTED>"
CLAUDE_MODEL = "claude-sonnet-4-5-20250929"  # Latest model
INPUT_FILE = Path("ballerina_grpo_X4.json")
OUTPUT_DIR = Path("corrected_completions")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Input file: {INPUT_FILE}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Model: {CLAUDE_MODEL}")


Input file: ballerina_grpo_X4.json
Output directory: corrected_completions
Model: claude-sonnet-4-5-20250929


In [2]:
# Load problems from JSON
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    problems = json.load(f)

print(f"Loaded {len(problems)} problems")


Loaded 627 problems


In [3]:
def parse_demo_input_output(prompt):
    """Extract test cases from Demo Input and Demo Output in prompt"""
    test_cases = []
    
    # Find Demo Input section
    demo_input_match = re.search(r'Demo Input:\s*\n(.*?)(?:\n\nDemo Output:|$)', prompt, re.DOTALL)
    demo_output_match = re.search(r'Demo Output:\s*\n(.*?)(?:\n\nNote:|$)', prompt, re.DOTALL)
    
    if demo_input_match and demo_output_match:
        try:
            # Parse the list format like ['4\\n', '27\\n']
            input_str = demo_input_match.group(1).strip()
            output_str = demo_output_match.group(1).strip()
            
            # Try to parse as Python list
            try:
                input_list = ast.literal_eval(input_str)
                output_list = ast.literal_eval(output_str)
            except:
                # If that fails, try to extract from string representation
                input_list = re.findall(r"'([^']*)'", input_str)
                output_list = re.findall(r"'([^']*)'", output_str)
            
            # Match inputs with outputs
            for i in range(min(len(input_list), len(output_list))):
                test_input = input_list[i].replace('\\n', '\n')
                test_output = output_list[i].replace('\\n', '\n')
                test_cases.append({
                    'input': test_input,
                    'output': test_output
                })
        except Exception as e:
            print(f"    Error parsing test cases: {e}")
            return []
    
    return test_cases

def format_error_feedback(error_details, validation_msg):
    """Format detailed error feedback for Claude"""
    feedback_parts = [validation_msg]
    
    if 'compilation_error' in error_details:
        feedback_parts.append("\n\n=== COMPILATION ERROR ===")
        feedback_parts.append("The code failed to compile. Here is the compilation error:")
        feedback_parts.append(f"\n{error_details['compilation_error']}")
        feedback_parts.append("\nPlease fix all compilation errors in your code.")
    
    if 'failing_test_case' in error_details:
        test_case = error_details['failing_test_case']
        feedback_parts.append("\n\n=== FAILING TEST CASE EXAMPLE ===")
        feedback_parts.append("Here is a test case that failed:")
        feedback_parts.append(f"\nTest Input:")
        feedback_parts.append(f"```\n{test_case['input']}\n```")
        feedback_parts.append(f"\nExpected Output:")
        feedback_parts.append(f"```\n{test_case['expected_output']}\n```")
        feedback_parts.append(f"\nActual Output (what your code produced):")
        feedback_parts.append(f"```\n{test_case['actual_output']}\n```")
        if 'error' in test_case:
            feedback_parts.append(f"\nError Type: {test_case['error']}")
        feedback_parts.append("\nPlease fix your code to produce the expected output for this test case.")
    
    return "\n".join(feedback_parts)


In [4]:
# System prompt for Ballerina code generation
SYSTEM_PROMPT = """You are a pragmatic Ballerina programmer who enjoys test driven development. Given the following question, write a Ballerina script to complete the task and then write the the unit tests to validate the functionality. Also implement a main function check with a user input.

1. Make the code simple and easy to understand.
2. Try to limit library usage to the standard library. Be careful with your types, and try to limit yourself to the basic built in types and standard library functions.
3. Before you start writing the function you can think through how to solve the problem and perform reasoning in the comments above the function.
4. Implement a main function to accept user inputs. Output the outputs of the function.
5. Don't define inputs yourself. Do not hardcode inputs. Only use user inputs.
6. Then write unit tests for the function you defined. Make sure to write at least 4 assertions to test the function. The tests should be a simple.

[Important] Strictly follow the following output format for each response: Make sure to include code inside <CODE> and <TESTS> blocks. Only use Ballerina Programming Language. Never use any other programming languages. Implement proper error handling.

# Overview
Brief overview about the solution.

<CODE>
```ballerina
// Reasoning goes here
// and can be multi-line

import ballerina/io;

function add(int a, int b) returns int {
    return a + b;
}

public function main() {
    string? input_line = io:readln();
    if input_line is string {
        string[] parts = input_line.split(" ");
        if parts.length() == 2 {
            int|error first_num = 'int:fromString(parts[0]);
            int|error second_num = 'int:fromString(parts[1]);

            if first_num is int && second_num is int {
                int result = add(first_num, second_num);
                io:println(result.toString());
            }
        }
    }
}
```
</CODE>

<TESTS>
```ballerina
import ballerina/test;

@test:Config { }
function testAssertEquals() {
    int addResult = add(40, 2);
    test:assertEquals(addResult, 42);

    addResult = add(0, 0);
    test:assertEquals(addResult, 0);

    addResult = add(-1, 1);
    test:assertEquals(addResult, 0);

    addResult = add(-5, -5);
    test:assertEquals(addResult, -10);
}
```
</TESTS>
"""

def generate_ballerina_solution(client, prompt, previous_attempt=None, error_info=None, error_details=None, attempt_num=1):
    """Generate Ballerina solution using Claude API"""
    
    if attempt_num == 1:
        # First attempt - use the original prompt with system prompt
        api_prompt = f"""{SYSTEM_PROMPT}

Problem:
{prompt}

Special Instructions:
Think Before You Speak: Remember that you operate under a strict causal restriction: once a token is produced, it is permanent. You cannot edit what you have already written. Therefore, you must "measure twice, cut once." strict foresight is required to prevent painting yourself into a corner. Use comments freely since it has no harm. 

Flexible Strategy: Abandon strict adherence to generic planning templates. Do not force every problem into a standard step-by-step list. Instead, assess the request and design a unique way to plan your solution. If a new, non-standard format helps you organize your thoughts better, use it.

Please generate Ballerina code that solves the problem above. Make sure to:
1. Read input from stdin (not command line arguments) using io:readln() or similar
2. Process the input according to the problem specification
3. Output the result to stdout using io:println()
4. Handle all edge cases properly
5. Use proper error handling
"""
    else:
        # Retry with detailed error feedback
        api_prompt = f"""{SYSTEM_PROMPT}

Problem:
{prompt}

Previous Attempt (FAILED):
{previous_attempt}

Detailed Error/Feedback:
{format_error_feedback(error_details or {}, error_info or "")}

Please generate corrected Ballerina code that:
1. Fixes the specific issues mentioned above (compilation errors, test case failures, etc.)
2. Follows the exact output format specified
3. Handles all edge cases properly
4. Uses proper error handling
5. Produces the expected output for the failing test case shown above
6. Reads input from stdin (not command line arguments) using io:readln() or similar
7. Think Before You Speak: Remember that you operate under a strict causal restriction: once a token is produced, it is permanent. You cannot edit what you have already written. Therefore, you must "measure twice, cut once." strict foresight is required to prevent painting yourself into a corner. Use comments freely since it has no harm. 
8. Flexible Strategy: Abandon strict adherence to generic planning templates. Do not force every problem into a standard step-by-step list. Instead, assess the request and design a unique way to plan your solution. If a new, non-standard format helps you organize your thoughts better, use it.

Return ONLY the code inside <CODE></CODE> and tests inside <TESTS></TESTS> blocks as specified in the original instructions."""
    
    try:
        message = client.messages.create(
            model=CLAUDE_MODEL,
            max_tokens=8000,
            messages=[{"role": "user", "content": api_prompt}]
        )
        
        response_text = message.content[0].text
        return response_text
        
    except Exception as e:
        print(f"  API Error: {e}")
        return None


In [5]:
def extract_code_from_completion(completion):
    """Extract code block from completion"""
    if '<CODE>' in completion and '</CODE>' in completion:
        code_section = completion.split('<CODE>')[1].split('</CODE>')[0]
        # Extract from ballerina code block
        if '```ballerina' in code_section:
            code = code_section.split('```ballerina')[1].split('```')[0].strip()
        elif '```' in code_section:
            code = code_section.split('```')[1].split('```')[0].strip()
        else:
            code = code_section.strip()
        return code
    return None


In [6]:
def normalize_output(output):
    """Normalize output by stripping trailing whitespace from each line and the entire output"""
    lines = output.split('\n')
    normalized_lines = [line.rstrip() for line in lines]
    return '\n'.join(normalized_lines).strip()

def run_test_cases(code, test_cases, threshold=0.75):
    """Run test cases and check if at least threshold% pass
    Returns: (is_valid, passed, total, validation_msg, error_details)
    error_details includes: compilation_error, failing_test_case
    """
    if not code:
        return False, 0, 0, "No code extracted", {}
    
    # Check for basic Ballerina structure
    if 'function main()' not in code and 'public function main()' not in code:
        return False, 0, 0, "Missing main function", {}
    
    error_details = {}
    
    if not test_cases or len(test_cases) == 0:
        # No test cases, fall back to compilation check
        with tempfile.TemporaryDirectory() as tmpdir:
            code_file = Path(tmpdir) / "main.bal"
            code_file.write_text(code)
            
            try:
                result = subprocess.run(
                    ["bal", "build", "main.bal"],
                    cwd=tmpdir,
                    capture_output=True,
                    text=True,
                    timeout=30
                )
                
                if result.returncode == 0:
                    return True, 0, 0, "Code compiles (no test cases)", {}
                else:
                    error_details['compilation_error'] = result.stderr
                    return False, 0, 0, f"Compilation error: {result.stderr[:500]}", error_details
            except Exception as e:
                error_details['compilation_error'] = str(e)
                return False, 0, 0, f"Validation error: {str(e)}", error_details
    
    # First, try to compile the code
    with tempfile.TemporaryDirectory() as tmpdir:
        code_file = Path(tmpdir) / "main.bal"
        code_file.write_text(code)
        
        # Check compilation first
        compile_result = subprocess.run(
            ["bal", "build", "main.bal"],
            cwd=tmpdir,
            capture_output=True,
            text=True,
            timeout=30
        )
        
        if compile_result.returncode != 0:
            error_details['compilation_error'] = compile_result.stderr
            return False, 0, len(test_cases), f"Compilation error: {compile_result.stderr[:500]}", error_details
        
        # Code compiles, now run test cases
        passed = 0
        failed = 0
        first_failing_test = None
        
        for idx, test_case in enumerate(test_cases):
            test_input = test_case.get('input', '')
            expected_output = test_case.get('output', '')
            
            # Normalize expected output
            expected_output_normalized = normalize_output(expected_output)
            
            try:
                # Write input to temp file
                input_file = Path(tmpdir) / f"input_{idx}.txt"
                input_file.write_text(test_input)
                
                # Run the code
                with open(input_file, 'r') as inp:
                    result = subprocess.run(
                        ["bal", "run", "main.bal"],
                        cwd=tmpdir,
                        stdin=inp,
                        capture_output=True,
                        text=True,
                        timeout=10
                    )
                
                if result.returncode != 0:
                    failed += 1
                    # Capture first failing test with runtime error
                    if first_failing_test is None:
                        first_failing_test = {
                            'input': test_input,
                            'expected_output': expected_output,
                            'actual_output': result.stderr if result.stderr else result.stdout,
                            'error': f"Runtime error (exit code {result.returncode})"
                        }
                    continue
                
                # Normalize actual output
                actual_output_normalized = normalize_output(result.stdout)
                
                if actual_output_normalized == expected_output_normalized:
                    passed += 1
                else:
                    failed += 1
                    # Capture first failing test with output mismatch
                    if first_failing_test is None:
                        first_failing_test = {
                            'input': test_input,
                            'expected_output': expected_output,
                            'actual_output': result.stdout,
                            'error': 'Output mismatch'
                        }
                    
            except subprocess.TimeoutExpired:
                failed += 1
                if first_failing_test is None:
                    first_failing_test = {
                        'input': test_input,
                        'expected_output': expected_output,
                        'actual_output': '',
                        'error': 'Timeout (code took >10 seconds)'
                    }
            except Exception as e:
                failed += 1
                if first_failing_test is None:
                    first_failing_test = {
                        'input': test_input,
                        'expected_output': expected_output,
                        'actual_output': '',
                        'error': str(e)
                    }
        
        total = len(test_cases)
        pass_rate = passed / total if total > 0 else 0
        
        if pass_rate >= threshold:
            return True, passed, total, f"Passed {passed}/{total} tests ({pass_rate*100:.1f}%)", {}
        else:
            if first_failing_test:
                error_details['failing_test_case'] = first_failing_test
            return False, passed, total, f"Only passed {passed}/{total} tests ({pass_rate*100:.1f}%), need {threshold*100:.0f}%", error_details


In [7]:
def process_problem(problem, client, problem_idx):
    """Process a single problem with retry logic - records all attempts"""
    
    # Create unique ID from problem index
    entry_id = f"problem_{problem_idx:04d}"
    output_file = OUTPUT_DIR / f"{entry_id}.json"
    failed_file = OUTPUT_DIR / f"{entry_id}_failed.json"
    
    # Check if already processed (resuming)
    if output_file.exists() or failed_file.exists():
        print(f"  ⏭ Already processed, skipping")
        return 'skipped'
    
    prompt = problem.get('prompt', '')
    rating = problem.get('rating', 'unknown')
    
    # Extract test cases from prompt
    test_cases = parse_demo_input_output(prompt)
    if test_cases:
        print(f"  📋 {len(test_cases)} test cases extracted")
    else:
        print(f"  ⚠ No test cases found in prompt")
    
    max_attempts = 2
    previous_completion = None
    previous_error_details = {}
    error_info = "Initial generation attempt"
    
    # Track all attempts
    all_attempts = []
    final_status = 'failed'
    
    for attempt in range(1, max_attempts + 1):
        print(f"  Attempt {attempt}/{max_attempts}...")
        attempt_start_time = datetime.now()
        
        attempt_record = {
            'attempt_number': attempt,
            'timestamp': attempt_start_time.isoformat(),
            'status': 'failed',
            'completion': None,
            'code': None,
            'tests_passed': 0,
            'tests_total': len(test_cases) if test_cases else 0,
            'tests_pass_rate': 0.0,
            'validation_msg': None,
            'error': None,
            'error_details': {}
        }
        
        if attempt == 1:
            # First attempt
            completion = generate_ballerina_solution(client, prompt, attempt_num=1)
        else:
            # Retry with detailed feedback
            completion = generate_ballerina_solution(
                client, 
                prompt, 
                previous_attempt=previous_completion,
                error_info=error_info,
                error_details=previous_error_details,
                attempt_num=attempt
            )
        
        attempt_record['completion'] = completion
        
        if not completion:
            error_info = "API call failed"
            attempt_record['error'] = error_info
            attempt_record['validation_msg'] = error_info
            previous_completion = completion
            all_attempts.append(attempt_record)
            continue
        
        # Extract and validate code
        code = extract_code_from_completion(completion)
        attempt_record['code'] = code
        
        if not code:
            error_info = "Could not extract code from completion"
            attempt_record['error'] = error_info
            attempt_record['validation_msg'] = error_info
            previous_completion = completion
            all_attempts.append(attempt_record)
            continue
        
        # Run test cases (75% threshold) - now returns error_details
        is_valid, passed, total, validation_msg, error_details = run_test_cases(code, test_cases, threshold=0.75)
        
        attempt_record['tests_passed'] = passed
        attempt_record['tests_total'] = total
        attempt_record['tests_pass_rate'] = passed/total if total > 0 else 0.0
        attempt_record['validation_msg'] = validation_msg
        attempt_record['error_details'] = error_details
        
        if is_valid:
            # Success!
            attempt_record['status'] = 'success'
            final_status = 'success'
            all_attempts.append(attempt_record)
            
            # Save the result with all attempts
            result = {
                'entry_id': entry_id,
                'problem_index': problem_idx,
                'prompt': prompt,
                'rating': rating,
                'final_status': 'success',
                'successful_attempt': attempt,
                'completion': completion,
                'code': code,
                'tests_passed': passed,
                'tests_total': total,
                'tests_pass_rate': passed/total if total > 0 else 0,
                'validation_msg': validation_msg,
                'all_attempts': all_attempts,
                'total_attempts': len(all_attempts),
                'timestamp': datetime.now().isoformat()
            }
            
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(result, f, indent=2, ensure_ascii=False)
            
            print(f"  ✓ Success on attempt {attempt}: {validation_msg}")
            return 'success'
        else:
            # Failed attempt - save error details for next retry
            attempt_record['status'] = 'failed'
            attempt_record['error'] = validation_msg
            error_info = validation_msg
            previous_completion = completion
            previous_error_details = error_details  # Store for next attempt
            all_attempts.append(attempt_record)
            print(f"  ✗ Attempt {attempt} failed: {validation_msg}")
    
    # All attempts failed - save with all attempt records
    print(f"  ✗ All {max_attempts} attempts failed")
    
    failure_info = {
        'entry_id': entry_id,
        'problem_index': problem_idx,
        'prompt': prompt,
        'rating': rating,
        'final_status': 'failed',
        'all_attempts': all_attempts,
        'total_attempts': len(all_attempts),
        'last_error': error_info,
        'timestamp': datetime.now().isoformat()
    }
    
    failure_file = OUTPUT_DIR / f"{entry_id}_failed.json"
    with open(failure_file, 'w', encoding='utf-8') as f:
        json.dump(failure_info, f, indent=2, ensure_ascii=False)
    
    return 'failed'


In [8]:
# Initialize Claude client
if not CLAUDE_API_KEY:
    print("⚠ WARNING: ANTHROPIC_API_KEY environment variable not set!")
    print("Please set it with: export ANTHROPIC_API_KEY='your-key-here'")
else:
    client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)
    print("✓ Claude client initialized")


✓ Claude client initialized


In [9]:
# Process all problems
print("=" * 80)
print(f"Starting processing of {len(problems)} problems")
print("=" * 80)

stats = {
    'success': 0,
    'failed': 0,
    'skipped': 0
}

for idx, problem in enumerate(problems):
    print(f"\n[{idx + 1}/{len(problems)}] Processing problem {idx} (rating: {problem.get('rating', 'unknown')})...")
    
    result = process_problem(problem, client, idx)
    stats[result] += 1
    
    # Print progress every 10 entries
    if (idx + 1) % 10 == 0:
        print("\n" + "-" * 80)
        print(f"Progress: {idx + 1}/{len(problems)}")
        print(f"Success: {stats['success']}, Failed: {stats['failed']}, Skipped: {stats['skipped']}")
        print("-" * 80)

# Final summary
print("\n" + "=" * 80)
print("PROCESSING COMPLETE")
print("=" * 80)
print(f"Total problems: {len(problems)}")
print(f"Successful generations: {stats['success']}")
print(f"Failed after retries: {stats['failed']}")
print(f"Skipped (already processed): {stats['skipped']}")
processed = len(problems) - stats['skipped']
if processed > 0:
    print(f"Success rate: {stats['success']/processed*100:.1f}%")
print("=" * 80)


Starting processing of 627 problems

[1/627] Processing problem 0 (rating: 1600)...
  📋 2 test cases extracted
  Attempt 1/2...
  ✓ Success on attempt 1: Passed 2/2 tests (100.0%)

[2/627] Processing problem 1 (rating: 1600)...
  📋 3 test cases extracted
  Attempt 1/2...
  ✗ Attempt 1 failed: Compilation error: ERROR [main.bal:(81:35,81:40)] undefined function 'split' in type 'string'
error: compilation contains errors

  Attempt 2/2...
  ✗ Attempt 2 failed: Only passed 0/3 tests (0.0%), need 75%
  ✗ All 2 attempts failed

[3/627] Processing problem 2 (rating: 1800)...
  📋 3 test cases extracted
  Attempt 1/2...
  ✗ Attempt 1 failed: Compilation error: ERROR [main.bal:(43:13,43:17)] invalid token 'from'
ERROR [main.bal:(43:18,43:18)] missing identifier
ERROR [main.bal:(43:20,43:28)] incompatible types: expected 'string:Char', found 'string'
ERROR [main.bal:(43:20,43:46)] too many arguments in call to 'toCodePointInt()'
ERROR [main.bal:(43:49,43:70)] too many arguments in call to 'toCod